# Example Data Notebook for TAQ Database


In [30]:
from scripts.query_helpers import client_connection
import pandas as pd

### Personal Details

Below are the details needed to authenticate into the server and crete a client connection

In [31]:
host="ppolak5.ams.stonybrook.edu."
server_user= 'TEMP'
server_password='TEMP'
db_user='TEMP'
db_pass='TEMP'

In [32]:
conn = client_connection(
   host,server_user,server_password,db_user,db_pass
)

### Example Trade Query

In [34]:
exchange="K"
symbol='AAPL'
start= "2020-01-04"
end="2020-01-11"


In [35]:
result=conn.client_get_trades(exchange,symbol,start,end)

INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.2p1)


Trade Query for K AAPL 2020-01-04 2020-01-11


INFO:paramiko.transport:Authentication (password) successful!
/home/jbohne/.conda/envs/query_user/lib/python3.10/site-packages/clickhouse_sqlalchemy/drivers/base.py:268: SAWarning: Did not recognize type 'DateTime64(9, 'America/New_York')' of column 'Time'
  warn("Did not recognize type '%s' of column '%s'" %
INFO:paramiko.transport.sftp:[chan 1] Opened sftp connection (server version 3)


In [27]:
pd.read_csv('data/AAPL_trades.csv')

,Time,Exchange,Symbol,Trade_Volume,Trade_Price,Sale_Condition,Source_of_Trade
0,2020-01-06 07:00:00.022403,K,AAPL,5,294.54,@ TI,N
1,2020-01-06 07:00:00.022597,K,AAPL,5,294.54,@ TI,N
2,2020-01-06 07:00:00.025523,K,AAPL,5,294.71,@ TI,N
3,2020-01-06 07:00:00.025578,K,AAPL,10,294.71,@ TI,N
4,2020-01-06 07:00:00.025595,K,AAPL,10,294.71,@ TI,N
...,...,...,...,...,...,...,...
76888,2020-01-09 19:59:53.972777,K,AAPL,5,310.40,@ TI,N
76889,2020-01-09 19:59:53.972780,K,AAPL,300,310.40,@ T,N
76890,2020-01-09 19:59:53.972787,K,AAPL,170,310.40,@ T,N
76891,2020-01-09 19:59:55.325904,K,AAPL,25,310.40,@ TI,N


### Example Quote Query

In [36]:
exchange="K"
symbol='AAPL'
start= "2020-01-04"
end="2020-01-11"


In [ ]:
conn = client_connection(
   host,server_user,server_password,db_user,db_pass
)

In [39]:
result=conn.client_get_quotes(exchange,symbol,start,end)

INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.2p1)


Quote Query for K AAPL 2020-01-04 2020-01-11


INFO:paramiko.transport:Authentication (password) successful!
INFO:paramiko.transport.sftp:[chan 1] Opened sftp connection (server version 3)


In [40]:
pd.read_csv('data/AAPL_quotes.csv')

,Time,Exchange,Symbol,Bid_Price,Bid_Size,Offer_Price,Offer_Size,Best_Bid_Exchange,Best_Bid_Price,Best_Bid_Size,Best_Offer_Exchange,Best_Offer_Price,Best_Offer_Size
0,2020-01-06 07:00:00.026517,K,AAPL,294.29,4.0,294.50,2.0,P,294.48,1.0,K,294.50,2.0
1,2020-01-06 07:00:00.026576,K,AAPL,294.29,4.0,296.99,2.0,P,294.48,1.0,P,294.75,2.0
2,2020-01-06 07:00:00.027372,K,AAPL,294.29,4.0,294.75,5.0,P,294.48,1.0,K,294.75,5.0
3,2020-01-06 07:00:00.028759,K,AAPL,294.40,1.0,294.50,53.0,P,294.48,1.0,K,294.50,53.0
4,2020-01-06 07:00:00.028855,K,AAPL,294.40,1.0,294.50,52.0,P,294.48,1.0,K,294.50,52.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95316,2020-01-10 19:58:43.961198,K,AAPL,310.70,1.0,310.80,13.0,K,310.70,1.0,P,310.70,22.0
95317,2020-01-10 19:58:43.961210,K,AAPL,310.65,3.0,310.80,13.0,K,310.65,3.0,P,310.70,22.0
95318,2020-01-10 19:58:54.236415,K,AAPL,310.65,1.0,310.80,13.0,Q,310.65,1.0,P,310.70,21.0
95319,2020-01-10 19:59:10.797992,K,AAPL,310.65,40.0,310.80,13.0,K,310.65,40.0,P,310.70,21.0


### Notes
- You need to reauthenticate the connection for each query